# ECG Image Digitization - Inference Only

This notebook loads a pre-trained model and generates predictions for the PhysioNet Challenge.

**Competition**: [PhysioNet ECG Image Digitization](https://www.kaggle.com/competitions/physionet-ecg-image-digitization)

## Pipeline Overview
1. ✅ Environment Setup
2. ✅ Load Pre-trained Model
3. ✅ Run Inference
4. ✅ Generate Submission

## 1. Environment Setup

In [ ]:
import os
import sys
from pathlib import Path

# Detect Kaggle environment
IS_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ
print(f"Running on Kaggle: {IS_KAGGLE}")

# Install dependencies if needed
if IS_KAGGLE:
    !pip install -q "numpy<2" "pandas>=2.2.2" segmentation-models-pytorch wfdb
    
    # Paths for Kaggle
    DATA_DIR = Path('/kaggle/input/physionet-ecg-image-digitization')
    OUTPUT_DIR = Path('/kaggle/working')
    # Model is in the git repo attached as dataset
    MODEL_DIR = Path('/kaggle/input/physionet-ecg-models')  # Your repo attached as dataset
else:
    # Local paths
    DATA_DIR = Path('../data')
    OUTPUT_DIR = Path('../outputs')
    MODEL_DIR = Path('../models/exports')

print(f"Data directory: {DATA_DIR}")
print(f"Model directory: {MODEL_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

In [ ]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import json

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\n🖥️ Device: {device}")

if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Load Pre-trained Model

In [ ]:
# Add src to path
if not IS_KAGGLE:
    src_path = Path('../src')
    if src_path.exists() and str(src_path) not in sys.path:
        sys.path.insert(0, str(src_path.resolve()))

# Import model components
from ecg_digitization.models import ECGDigitizer
from ecg_digitization.data import get_val_transforms

# Model configuration (should match training)
MODEL_CONFIG = {
    'encoder_name': 'resnet50',
    'encoder_weights': None,  # Will load from checkpoint
    'num_leads': 12,
    'signal_length': 5000,
}

# Image configuration
IMAGE_SIZE = (512, 640)

print("📋 Model Configuration:")
for k, v in MODEL_CONFIG.items():
    print(f"  {k}: {v}")
print(f"  image_size: {IMAGE_SIZE}")

In [ ]:
# Load model metadata if available
metadata_path = MODEL_DIR / 'model_metadata.json'
if metadata_path.exists():
    with open(metadata_path) as f:
        model_metadata = json.load(f)
    print("📊 Model Metadata:")
    for k, v in model_metadata.items():
        print(f"  {k}: {v}")
else:
    print("⚠️ No model metadata found")
    model_metadata = {}

In [ ]:
# Create model
print("\n🏗️ Creating model...")
model = ECGDigitizer(
    encoder_name=MODEL_CONFIG['encoder_name'],
    encoder_weights=MODEL_CONFIG['encoder_weights'],
    num_leads=MODEL_CONFIG['num_leads'],
    signal_length=MODEL_CONFIG['signal_length'],
)

# Load checkpoint
checkpoint_path = MODEL_DIR / 'ecg_digitizer.pt'
if not checkpoint_path.exists():
    # Try alternative paths
    alt_paths = [
        MODEL_DIR / 'best_model.pt',
        MODEL_DIR / 'checkpoints' / 'best_model.pt',
    ]
    for alt in alt_paths:
        if alt.exists():
            checkpoint_path = alt
            break

print(f"📁 Loading checkpoint: {checkpoint_path}")

if checkpoint_path.exists():
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    # Handle different checkpoint formats
    if isinstance(checkpoint, dict):
        if 'model_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['model_state_dict'])
        elif 'state_dict' in checkpoint:
            model.load_state_dict(checkpoint['state_dict'])
        else:
            model.load_state_dict(checkpoint)
    else:
        model.load_state_dict(checkpoint)
    
    print("✅ Model loaded successfully")
else:
    raise FileNotFoundError(f"Checkpoint not found: {checkpoint_path}")

# Move to device and set eval mode
model = model.to(device)
model.eval()

# Count parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"  Total parameters: {num_params:,}")

## 3. Prepare Test Data

In [ ]:
# Load test metadata
test_csv = DATA_DIR / 'test.csv'
test_df = pd.read_csv(test_csv)
print(f"📊 Test samples: {len(test_df)}")
print(f"Columns: {list(test_df.columns)}")
test_df.head()

In [ ]:
# Create transform
transform = get_val_transforms(IMAGE_SIZE)

# Find test images
test_dir = DATA_DIR / 'test'
test_images = list(test_dir.glob('*.png'))
print(f"\n📁 Found {len(test_images)} test images")

if len(test_images) > 0:
    print(f"  First image: {test_images[0].name}")

## 4. Run Inference

In [ ]:
def preprocess_image(image_path, transform):
    """Load and preprocess an image."""
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image=np.array(image))['image']
    return image_tensor

def run_inference(model, image_paths, transform, device, batch_size=8):
    """Run inference on a list of images."""
    model.eval()
    predictions = {}
    
    with torch.no_grad():
        for i in tqdm(range(0, len(image_paths), batch_size), desc="Inference"):
            batch_paths = image_paths[i:i + batch_size]
            batch_tensors = []
            
            for path in batch_paths:
                try:
                    tensor = preprocess_image(path, transform)
                    batch_tensors.append(tensor)
                except Exception as e:
                    print(f"Warning: Failed to process {path}: {e}")
                    continue
            
            if len(batch_tensors) == 0:
                continue
            
            # Stack batch
            batch = torch.stack(batch_tensors).to(device)
            
            # Forward pass
            outputs = model(batch, target_length=5000)
            signals = outputs['signals'].cpu().numpy()
            
            # Store predictions
            for j, path in enumerate(batch_paths[:len(signals)]):
                sample_id = path.stem  # Get filename without extension
                predictions[sample_id] = signals[j]
    
    return predictions

print("\n🔮 Running inference...")

In [ ]:
# Run inference
predictions = run_inference(
    model=model,
    image_paths=test_images,
    transform=transform,
    device=device,
    batch_size=8,
)

print(f"\n✅ Generated predictions for {len(predictions)} samples")

## 5. Generate Submission

In [ ]:
# Lead names for ECG
LEAD_NAMES = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']

def create_submission(predictions, lead_names=LEAD_NAMES):
    """Create submission DataFrame in required format."""
    rows = []
    
    for sample_id, signal in predictions.items():
        # Signal shape: [num_leads, signal_length]
        for lead_idx, lead_name in enumerate(lead_names):
            if lead_idx < signal.shape[0]:
                # Convert signal to list of values
                signal_values = signal[lead_idx].tolist()
                
                rows.append({
                    'id': f"{sample_id}_{lead_name}",
                    'signal': signal_values,
                })
    
    return pd.DataFrame(rows)

print("\n📝 Creating submission file...")

In [ ]:
# Create submission DataFrame
submission_df = create_submission(predictions)

print(f"Submission shape: {submission_df.shape}")
print(f"Columns: {list(submission_df.columns)}")
submission_df.head()

In [ ]:
# Save submission
submission_path = OUTPUT_DIR / 'submission.parquet'
submission_df.to_parquet(submission_path, index=False)

print(f"\n✅ Submission saved: {submission_path}")
print(f"  File size: {submission_path.stat().st_size / 1024 / 1024:.2f} MB")

## 6. Visualize Sample Predictions

In [ ]:
import matplotlib.pyplot as plt

# Visualize a few predictions
sample_ids = list(predictions.keys())[:3]

fig, axes = plt.subplots(len(sample_ids), 1, figsize=(14, 4 * len(sample_ids)))
if len(sample_ids) == 1:
    axes = [axes]

for idx, sample_id in enumerate(sample_ids):
    signal = predictions[sample_id]
    time = np.arange(signal.shape[1]) / 500  # 500 Hz sampling rate
    
    # Plot all 12 leads with offset
    for lead_idx in range(min(12, signal.shape[0])):
        axes[idx].plot(
            time, 
            signal[lead_idx, :] + lead_idx * 2, 
            linewidth=0.8, 
            alpha=0.8,
            label=LEAD_NAMES[lead_idx] if lead_idx < len(LEAD_NAMES) else f'Lead {lead_idx}'
        )
    
    axes[idx].set_xlabel('Time (s)', fontsize=11)
    axes[idx].set_ylabel('Lead (offset)', fontsize=11)
    axes[idx].set_title(f'Sample: {sample_id}', fontsize=12, fontweight='bold')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✅ Displayed {len(sample_ids)} sample predictions")

## 🎉 Submission Complete!

Your submission file has been generated:
- **File**: `submission.parquet`
- **Format**: Parquet with columns `id` and `signal`

### Next Steps
1. Submit to the [PhysioNet ECG Image Digitization competition](https://www.kaggle.com/competitions/physionet-ecg-image-digitization)
2. Check your leaderboard score

### Model Info
- **Architecture**: ECGDigitizer (ResNet50 encoder + Signal Regression Head)
- **Input**: ECG paper images (RGB)
- **Output**: 12-lead ECG signals (5000 samples each)